In [ ]:
!pip install q scipy==1.1.0
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torchvision
from time import time 
from torchvision import datasets, transforms
from torch import nn,optim 
import torch.nn.functional as F
from scipy.sparse.linalg import lsqr
from scipy.sparse import csr_matrix
from scipy.sparse import random
from scipy.sparse import rand
from scipy import stats
from torch.autograd import Variable


     |████████████████████████████████| 31.2MB 143kB/s 
ERROR: umap-learn 0.4.3 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,)), ])
trainset = datasets.MNIST('./',download = True, train = True, transform = transform)
testset = datasets.MNIST('./',download= True, train= False, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64,shuffle = True)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle = True)

dataiter = iter(trainloader)
images, labels = dataiter.next()

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [ ]:
#define CNN source = https://zhenye-na.github.io/2018/09/28/pytorch-cnn-cifar10.html
#padding same 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
#set random seed for reproducibility
torch.manual_seed(0)
net = Net()
torch.save(net.state_dict(),'weights_i.pth')

In [ ]:
#training
#hiumang
def one_hot_embedding(labels, num_classes):
    """Embedding labels to one-hot form.

    Args:
      labels: (LongTensor) class labels, sized [N,].
      num_classes: (int) number of classes.

    Returns:
      (tensor) encoded labels, sized [N, #classes].
    """
    y = torch.eye(num_classes) 
    return y[labels] 



criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.003, momentum = 0.9)
for epoch in range(20):  # loop over the dataset multiple times
    correct = 0
    total = 0
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # labelsOH = one_hot_embedding(labels,10).long()
        # labelsOH = labelsOH.to(cuda)
        # labels = labels.to(cuda)
        # zero the parameter gradients
        optimizer.zero_grad()

#         # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()        

        # print statistics
        running_loss += loss.item()
    print("Epoch Number: "+str(epoch+1)+'Training Accuracy: %d %%' % (
    100 * correct / total))

    # print("Epoch Number: "+str(epoch+1)+" Loss is: "+str(running_loss / 60000))
    # print('[%d] loss: %.3f' %(epoch + 1), (running_loss / 60000))

#save learned weights
torch.save(net.state_dict(),'weights_f.pth')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch Number: 1Training Accuracy: 68 %
Epoch Number: 2Training Accuracy: 89 %
Epoch Number: 3Training Accuracy: 91 %
Epoch Number: 4Training Accuracy: 92 %
Epoch Number: 5Training Accuracy: 93 %
Epoch Number: 6Training Accuracy: 94 %
Epoch Number: 7Training Accuracy: 94 %
Epoch Number: 8Training Accuracy: 94 %
Epoch Number: 9Training Accuracy: 94 %
Epoch Number: 10Training Accuracy: 95 %
Epoch Number: 11Training Accuracy: 95 %
Epoch Number: 12Training Accuracy: 95 %
Epoch Number: 13Training Accuracy: 95 %
Epoch Number: 14Training Accuracy: 95 %
Epoch Number: 15Training Accuracy: 95 %
Epoch Number: 16Training Accuracy: 96 %
Epoch Number: 17Training Accuracy: 96 %
Epoch Number: 18Training Accuracy: 96 %
Epoch Number: 19Training Accuracy: 96 %
Epoch Number: 20Training Accuracy: 96 %


In [ ]:
#testing #10 epochs
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
acc = (correct/total)
np.save('model_accuracy_3.npy',acc)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 10000 test images: 96 %


In [ ]:
#vectorize wi
wi =torch.load('weights_i.pth')

wi1 = np.asarray(wi['conv1.weight'].resize(1,10*1*5*5))
wi2 = np.asarray(wi['conv2.weight'].resize(1,20*10*5*5))
wi3 = np.asarray(wi['fc1.weight'].resize(1,320*50))  #size of the resulting image after the given layers of convolutions 
wi4 = np.asarray(wi['fc2.weight'].resize(1,50*10))  #size of the resulting image after the given layers of convolutions 

wi_vec = np.concatenate((wi1,wi2,wi3,wi4),axis =1)
wi_vec = wi_vec.T
dim_w = wi_vec.shape[0]
wi_vec.ravel().squeeze()

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


array([-0.00149736,  0.10728873, -0.16460903, ...,  0.04108326,
        0.04330994, -0.09308142], dtype=float32)

In [ ]:
#vectorize wf
wf  =torch.load('weights_f.pth')

wf1 = np.asarray(wf['conv1.weight'].resize(1,10*1*5*5))
wf2 = np.asarray(wf['conv2.weight'].resize(1,20*10*5*5))
wf3 = np.asarray(wf['fc1.weight'].resize(1,320*50))  #size of the resulting image after the given layers of convolutions 
wf4 = np.asarray(wf['fc2.weight'].resize(1,50*10))  #size of the resulting image after the given layers of convolutions 

wf_vec = np.concatenate((wf1,wf2,wf3,wf4),axis =1)
wf_vec = wf_vec.T
wf_vec.ravel().squeeze()

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


array([-0.03974413, -0.0800088 , -0.21986344, ...,  0.21692921,
       -0.07876498, -0.15856561], dtype=float32)

In [ ]:
dim_w

21750

In [ ]:
#initialize A run 1
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,21750])
n_runs = 2
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  320*50]
      w3 = torch.from_numpy(w3).reshape(50,320)
      w4 = w[10*1*5*5+20*10*5*5 +  320*50: 10*1*5*5+20*10*5*5 +  320*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.1141

 Dist from Wi = 0.04190422644438683 , Dist from Wf =11.106725833709449

 n_rows = 4 , Accuracy =0.1139

 Dist from Wi = 0.09293943833291025 , Dist from Wf =11.106416027231921

 n_rows = 8 , Accuracy =0.1138

 Dist from Wi = 0.19519730100574917 , Dist from Wf =11.105089496392749

 n_rows = 16 , Accuracy =0.1137

 Dist from Wi = 0.27285154359655733 , Dist from Wf =11.103452919926966

 n_rows = 32 , Accuracy =0.1141

 Dist from Wi = 0.43424567275296255 , Dist from Wf =11.098312727870507

 n_rows = 64 , Accuracy =0.114

 Dist from Wi = 0.5725214657754795 , Dist from Wf =11.09203921203685

 n_rows = 128 , Accuracy =0.1139

 Dist from Wi = 0.7646361863704307 , Dist from Wf =11.08045333963637

 n_rows = 256 , Accuracy =0.1144

 Dist from Wi = 1.1583311465728638 , Dist from Wf =11.046238439912358

 n_rows = 512 , Accuracy =0.115

 Dist from Wi = 1.647899007241548 , Dist from Wf =10.983876528066105

 n_rows = 1024 , Accuracy =0.1166

 Dist from Wi = 2.3405779276

In [ ]:
np.save("accuracy_A.npy", acc)
np.save("distance_i_A.npy", dist_i)
np.save("distance_f_A.npy", dist_f)

In [ ]:
#initialize A run 2
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384])
n_runs = 10
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  320*50]
      w3 = torch.from_numpy(w3).reshape(50,320)
      w4 = w[10*1*5*5+20*10*5*5 +  320*50: 10*1*5*5+20*10*5*5 +  320*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.1135

 Dist from Wi = 0.09652924531369025 , Dist from Wf =11.009491207705924

 n_rows = 4 , Accuracy =0.1135

 Dist from Wi = 0.1307251064672775 , Dist from Wf =11.009138272100522

 n_rows = 8 , Accuracy =0.1135

 Dist from Wi = 0.13560864024510635 , Dist from Wf =11.00907920056634

 n_rows = 16 , Accuracy =0.1135

 Dist from Wi = 0.23001294611206027 , Dist from Wf =11.007511462302537

 n_rows = 32 , Accuracy =0.1135

 Dist from Wi = 0.3325873801376039 , Dist from Wf =11.00488983061296

 n_rows = 64 , Accuracy =0.1135

 Dist from Wi = 0.5153257264304202 , Dist from Wf =10.9978476959132

 n_rows = 128 , Accuracy =0.1135

 Dist from Wi = 0.8790036654556294 , Dist from Wf =10.97476956945529

 n_rows = 256 , Accuracy =0.1135

 Dist from Wi = 1.249871815547243 , Dist from Wf =10.938740100801525

 n_rows = 512 , Accuracy =0.1135

 Dist from Wi = 1.7203623816523053 , Dist from Wf =10.874675527338256

 n_rows = 1024 , Accuracy =0.1137

 Dist from Wi = 2.3574022261253

In [ ]:
#initialize A run 3
from scipy import sparse
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384])
n_runs = 10
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )

  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  320*50]
      w3 = torch.from_numpy(w3).reshape(50,320)
      w4 = w[10*1*5*5+20*10*5*5 +  320*50: 10*1*5*5+20*10*5*5 +  320*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.1146

 Dist from Wi = 0.08410821895205219 , Dist from Wf =11.048431107217905

 n_rows = 4 , Accuracy =0.1155

 Dist from Wi = 0.1333340116122974 , Dist from Wf =11.047946694513646

 n_rows = 8 , Accuracy =0.1144

 Dist from Wi = 0.23421703258568505 , Dist from Wf =11.046268442661203

 n_rows = 16 , Accuracy =0.1142

 Dist from Wi = 0.33236734718906213 , Dist from Wf =11.043750996483455

 n_rows = 32 , Accuracy =0.1148

 Dist from Wi = 0.4487080511365565 , Dist from Wf =11.039636099559024

 n_rows = 64 , Accuracy =0.1138

 Dist from Wi = 0.6422976915347192 , Dist from Wf =11.030066083311542

 n_rows = 128 , Accuracy =0.113

 Dist from Wi = 0.8605831515899552 , Dist from Wf =11.015185008130107

 n_rows = 256 , Accuracy =0.1136

 Dist from Wi = 1.1905951116819873 , Dist from Wf =10.98441566077226

 n_rows = 512 , Accuracy =0.1148

 Dist from Wi = 1.7083988630557747 , Dist from Wf =10.915872729801324

 n_rows = 1024 , Accuracy =0.1174

 Dist from Wi = 2.371124064

In [ ]:
np.save("accuracy_3.npy", acc)
np.save("distance_i_3.npy", dist_i)
np.save("distance_f_3.npy", dist_f)

In [ ]:
#initialize A+I run 1
from scipy import sparse
I = sparse.identity(dim_w)
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,21750])
n_runs = 2
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )
  A_large = A_large + I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  320*50]
      w3 = torch.from_numpy(w3).reshape(50,320)
      w4 = w[10*1*5*5+20*10*5*5 +  320*50: 10*1*5*5+20*10*5*5 +  320*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.1138

 Dist from Wi = 0.1536095955548908 , Dist from Wf =11.105742604622561

 n_rows = 4 , Accuracy =0.1137

 Dist from Wi = 0.1669537372796397 , Dist from Wf =11.105550015992637

 n_rows = 8 , Accuracy =0.1141

 Dist from Wi = 0.3599294797118631 , Dist from Wf =11.10097137534893

 n_rows = 16 , Accuracy =0.1141

 Dist from Wi = 0.48038973449849565 , Dist from Wf =11.096411149882583

 n_rows = 32 , Accuracy =0.1144

 Dist from Wi = 0.6605051322729238 , Dist from Wf =11.08714785990257

 n_rows = 64 , Accuracy =0.1144

 Dist from Wi = 0.9093730035190463 , Dist from Wf =11.06951468857665

 n_rows = 128 , Accuracy =0.1156

 Dist from Wi = 1.2948977468735363 , Dist from Wf =11.031063164539386

 n_rows = 256 , Accuracy =0.116

 Dist from Wi = 1.6351098698458484 , Dist from Wf =10.985787656118525

 n_rows = 512 , Accuracy =0.124

 Dist from Wi = 2.0455254607006474 , Dist from Wf =10.916819147707672

 n_rows = 1024 , Accuracy =0.1297

 Dist from Wi = 2.68407282131467

In [ ]:
np.save("accuracy_AI.npy", acc)
np.save("distance_i_AI.npy", dist_i)
np.save("distance_f_AI.npy", dist_f)

In [ ]:
#initialize I run 1
from scipy import sparse
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,10*25,250+200*25,250+200*25+320*50,250+200*25+320*50+500])
num_rows_arr = np.sort(num_rows_arr)
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)

      w = np.zeros((dim_w,1))
      w[0:n_rows] = wf_vec[0:n_rows]
      w[n_rows:] = wi_vec[n_rows:]

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  320*50]
      w3 = torch.from_numpy(w3).reshape(50,320)
      w4 = w[10*1*5*5+20*10*5*5 +  320*50: 10*1*5*5+20*10*5*5 +  320*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.1142

 Dist from Wi = 0.19116270633936885 , Dist from Wf =11.105159680484038

 n_rows = 4 , Accuracy =0.1136

 Dist from Wi = 0.24171687807299364 , Dist from Wf =11.104174334915585

 n_rows = 8 , Accuracy =0.1144

 Dist from Wi = 0.7593146857776234 , Dist from Wf =11.08081928005803

 n_rows = 16 , Accuracy =0.1161

 Dist from Wi = 1.1783141248384479 , Dist from Wf =11.044124706492115

 n_rows = 32 , Accuracy =0.1215

 Dist from Wi = 1.5840754765022038 , Dist from Wf =10.993262463622328

 n_rows = 64 , Accuracy =0.1244

 Dist from Wi = 1.8049259848416996 , Dist from Wf =10.95916771012227

 n_rows = 128 , Accuracy =0.119

 Dist from Wi = 2.4322487279274747 , Dist from Wf =10.837217393539659

 n_rows = 250 , Accuracy =0.1284

 Dist from Wi = 3.3217602365552263 , Dist from Wf =10.598444397183197

 n_rows = 256 , Accuracy =0.1263

 Dist from Wi = 3.3352308059214693 , Dist from Wf =10.594213051500134

 n_rows = 512 , Accuracy =0.1494

 Dist from Wi = 3.538495426286

In [ ]:
np.save("accuracy_I.npy", acc)
np.save("distance_i_I.npy", dist_i)
np.save("distance_f_I.npy", dist_f)

In [ ]:
#initialize Irev run 1
from scipy import sparse
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384,10*25,250+200*25,250+200*25+320*50,250+200*25+320*50+500])
num_rows_arr = np.sort(num_rows_arr)
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)

      w = np.zeros((dim_w,1))
      w[0:-n_rows] = wi_vec[0:-n_rows]
      w[-n_rows:] = wf_vec[-n_rows:]  

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  320*50]
      w3 = torch.from_numpy(w3).reshape(50,320)
      w4 = w[10*1*5*5+20*10*5*5 +  320*50: 10*1*5*5+20*10*5*5 +  320*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.1139

 Dist from Wi = 0.13852965131559172 , Dist from Wf =11.105940943704397

 n_rows = 4 , Accuracy =0.114

 Dist from Wi = 0.22607218709537558 , Dist from Wf =11.104503864449113

 n_rows = 8 , Accuracy =0.114

 Dist from Wi = 0.429629699696575 , Dist from Wf =11.098492376466362

 n_rows = 16 , Accuracy =0.1137

 Dist from Wi = 0.6245825136356356 , Dist from Wf =11.089229522063466

 n_rows = 32 , Accuracy =0.115

 Dist from Wi = 1.0024432591147852 , Dist from Wf =11.061474685664596

 n_rows = 64 , Accuracy =0.1185

 Dist from Wi = 1.3289236675699836 , Dist from Wf =11.027015761080406

 n_rows = 128 , Accuracy =0.12

 Dist from Wi = 1.7447062057149345 , Dist from Wf =10.96891585185533

 n_rows = 250 , Accuracy =0.1201

 Dist from Wi = 2.6692523823668077 , Dist from Wf =10.781289645889887

 n_rows = 256 , Accuracy =0.1191

 Dist from Wi = 2.6851345934674775 , Dist from Wf =10.777345077722417

 n_rows = 512 , Accuracy =0.1222

 Dist from Wi = 3.854626812169523 

In [ ]:
np.save("accuracy_UI.npy", acc)
np.save("distance_i_UI.npy", dist_i)
np.save("distance_f_UI.npy", dist_f)

In [ ]:
from google.colab import files
files.download('accuracy_UI.npy') 
files.download('distance_i_UI.npy')
files.download('distance_f_UI.npy')

files.download('accuracy_I.npy') 
files.download('distance_i_I.npy')
files.download('distance_f_I.npy')

files.download('accuracy_AI.npy') 
files.download('distance_i_AI.npy')
files.download('distance_f_AI.npy')

files.download('accuracy_A.npy') 
files.download('distance_i_A.npy')
files.download('distance_f_A.npy')

In [ ]:
#initialize A+I run 2
from scipy import sparse
I = sparse.identity(dim_w)
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384])
n_runs = 10
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )
  A_large = A_large + I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  320*50]
      w3 = torch.from_numpy(w3).reshape(50,320)
      w4 = w[10*1*5*5+20*10*5*5 +  320*50: 10*1*5*5+20*10*5*5 +  320*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.1135

 Dist from Wi = 0.15765634396011222 , Dist from Wf =11.008785538253477

 n_rows = 4 , Accuracy =0.1135

 Dist from Wi = 0.16527042733455444 , Dist from Wf =11.008673863621713

 n_rows = 8 , Accuracy =0.1135

 Dist from Wi = 0.25628456302808716 , Dist from Wf =11.006931124166787

 n_rows = 16 , Accuracy =0.1135

 Dist from Wi = 0.3498481134744893 , Dist from Wf =11.004354631144677

 n_rows = 32 , Accuracy =0.1135

 Dist from Wi = 0.46550448156167457 , Dist from Wf =11.000069096471222

 n_rows = 64 , Accuracy =0.1135

 Dist from Wi = 0.7412176902372881 , Dist from Wf =10.98493563382768

 n_rows = 128 , Accuracy =0.1135

 Dist from Wi = 1.0907043621266599 , Dist from Wf =10.955755498579062

 n_rows = 256 , Accuracy =0.1135

 Dist from Wi = 1.5326473644539238 , Dist from Wf =10.902715561190819

 n_rows = 512 , Accuracy =0.1135

 Dist from Wi = 1.9621410009047258 , Dist from Wf =10.833661303855168

 n_rows = 1024 , Accuracy =0.1135

 Dist from Wi = 2.6059078

In [ ]:
#initialize A+I run 3
from scipy import sparse
I = sparse.identity(dim_w)
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384])
n_runs = 10
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large = sparse.random(dim_w,dim_w, format='csr', density=den, data_rvs = rvs )
  A_large = A_large + I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  320*50]
      w3 = torch.from_numpy(w3).reshape(50,320)
      w4 = w[10*1*5*5+20*10*5*5 +  320*50: 10*1*5*5+20*10*5*5 +  320*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.1151

 Dist from Wi = 0.060170445276240846 , Dist from Wf =11.048587404795546

 n_rows = 4 , Accuracy =0.1156

 Dist from Wi = 0.3747229647395466 , Dist from Wf =11.042394976630032

 n_rows = 8 , Accuracy =0.1155

 Dist from Wi = 0.49206824419330625 , Dist from Wf =11.03778840913571

 n_rows = 16 , Accuracy =0.1161

 Dist from Wi = 0.615096023172135 , Dist from Wf =11.03161642759178

 n_rows = 32 , Accuracy =0.1156

 Dist from Wi = 0.7013228120965282 , Dist from Wf =11.026470443186293

 n_rows = 64 , Accuracy =0.1165

 Dist from Wi = 0.8805358108411089 , Dist from Wf =11.013607982835367

 n_rows = 128 , Accuracy =0.1181

 Dist from Wi = 1.1829472912275834 , Dist from Wf =10.985241909734077

 n_rows = 256 , Accuracy =0.1239

 Dist from Wi = 1.5874691763520266 , Dist from Wf =10.934113851699584

 n_rows = 512 , Accuracy =0.1231

 Dist from Wi = 2.0082141672676292 , Dist from Wf =10.864712604160083

 n_rows = 1024 , Accuracy =0.1279

 Dist from Wi = 2.6208276759

In [ ]:
np.save("accuracy_AI3.npy", acc)
np.save("distance_i_AI3.npy", dist_i)
np.save("distance_f_AI3.npy", dist_f)

In [ ]:
#initialize I run 1
from scipy import sparse
I = sparse.identity(dim_w)
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384])
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large =  I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  320*50]
      w3 = torch.from_numpy(w3).reshape(50,320)
      w4 = w[10*1*5*5+20*10*5*5 +  320*50: 10*1*5*5+20*10*5*5 +  320*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.1141

 Dist from Wi = 0.24356717497505406 , Dist from Wf =10.996374186592666

 n_rows = 4 , Accuracy =0.114

 Dist from Wi = 0.32613292649518116 , Dist from Wf =10.994235195525995

 n_rows = 8 , Accuracy =0.1146

 Dist from Wi = 0.5608448273013958 , Dist from Wf =10.984763233678555

 n_rows = 16 , Accuracy =0.117

 Dist from Wi = 0.8497941939895521 , Dist from Wf =10.96619441963148

 n_rows = 32 , Accuracy =0.1224

 Dist from Wi = 1.2691308769665839 , Dist from Wf =10.925606484288938

 n_rows = 64 , Accuracy =0.1311

 Dist from Wi = 1.654665842563247 , Dist from Wf =10.873897698541818

 n_rows = 128 , Accuracy =0.1245

 Dist from Wi = 2.4168479969098207 , Dist from Wf =10.73025703056786

 n_rows = 256 , Accuracy =0.1263

 Dist from Wi = 3.1458023093914065 , Dist from Wf =10.539615645453527

 n_rows = 512 , Accuracy =0.124

 Dist from Wi = 3.38354761630856 , Dist from Wf =10.465714291622678

 n_rows = 1024 , Accuracy =0.1385

 Dist from Wi = 3.868390321669809 

In [ ]:
#initialize I run 2
from scipy import sparse
I = sparse.identity(dim_w)
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384])
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large =  I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  320*50]
      w3 = torch.from_numpy(w3).reshape(50,320)
      w4 = w[10*1*5*5+20*10*5*5 +  320*50: 10*1*5*5+20*10*5*5 +  320*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.1135

 Dist from Wi = 0.1885721375138103 , Dist from Wf =11.008299373514584

 n_rows = 4 , Accuracy =0.1135

 Dist from Wi = 0.32597561632231775 , Dist from Wf =11.005087661874002

 n_rows = 8 , Accuracy =0.1135

 Dist from Wi = 0.5009313471560757 , Dist from Wf =10.99851273287767

 n_rows = 16 , Accuracy =0.1136

 Dist from Wi = 0.9484828765875242 , Dist from Wf =10.96898330719343

 n_rows = 32 , Accuracy =0.1137

 Dist from Wi = 1.0633221860204678 , Dist from Wf =10.95844699271394

 n_rows = 64 , Accuracy =0.115

 Dist from Wi = 1.4073211583390923 , Dist from Wf =10.919599887983345

 n_rows = 128 , Accuracy =0.1213

 Dist from Wi = 2.307581380378736 , Dist from Wf =10.765374251752728

 n_rows = 256 , Accuracy =0.1202

 Dist from Wi = 3.2479464969198304 , Dist from Wf =10.519936223856297

 n_rows = 512 , Accuracy =0.1175

 Dist from Wi = 3.4928772251741993 , Dist from Wf =10.441169631854185

 n_rows = 1024 , Accuracy =0.1133

 Dist from Wi = 3.87180488103195

In [ ]:
#initialize I run 3
from scipy import sparse
I = sparse.identity(dim_w)
rvs = stats.norm().rvs
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384])
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  A_large =  I
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)
      A = A_large.tocsr()[0:n_rows,:]
      w = np.zeros(dim_w)


      w_diff = csr_matrix(wf_vec - wi_vec)
      b_eq = sparse.csr_matrix.dot(A,w_diff)

      bn = b_eq.toarray()
      bn.reshape((-1,))
      
      sol = lsqr(A,bn)
      z = sol[0]
      z = np.asmatrix(z).T
      w = z+ wi_vec

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  320*50]
      w3 = torch.from_numpy(w3).reshape(50,320)
      w4 = w[10*1*5*5+20*10*5*5 +  320*50: 10*1*5*5+20*10*5*5 +  320*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.1146

 Dist from Wi = 0.13281575191601008 , Dist from Wf =11.047952937076563

 n_rows = 4 , Accuracy =0.1147

 Dist from Wi = 0.5366500628112321 , Dist from Wf =11.035710707445574

 n_rows = 8 , Accuracy =0.1141

 Dist from Wi = 1.036618067139996 , Dist from Wf =11.000014867800054

 n_rows = 16 , Accuracy =0.1153

 Dist from Wi = 1.4162865216644795 , Dist from Wf =10.957601773423532

 n_rows = 32 , Accuracy =0.1162

 Dist from Wi = 1.5567563602420675 , Dist from Wf =10.938528866410142

 n_rows = 64 , Accuracy =0.1219

 Dist from Wi = 1.7880983485243114 , Dist from Wf =10.903100861508975

 n_rows = 128 , Accuracy =0.1185

 Dist from Wi = 2.442910223609733 , Dist from Wf =10.775300169152787

 n_rows = 256 , Accuracy =0.1228

 Dist from Wi = 3.2063443413404875 , Dist from Wf =10.573280477590028

 n_rows = 512 , Accuracy =0.1253

 Dist from Wi = 3.4539628980473482 , Dist from Wf =10.495000923604852

 n_rows = 1024 , Accuracy =0.1155

 Dist from Wi = 3.83151623706

In [ ]:
np.save("accuracy_I3.npy", acc)
np.save("distance_i_I3.npy", dist_i)
np.save("distance_f_I3.npy", dist_f)

In [ ]:
# I UI experiment

In [ ]:
!pip install q scipy==1.1.0
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torchvision
from time import time 
from torchvision import datasets, transforms
from torch import nn,optim 
import torch.nn.functional as F
from scipy.sparse.linalg import lsqr
from scipy.sparse import csr_matrix
from scipy.sparse import random
from scipy.sparse import rand
from scipy import stats
from torch.autograd import Variable


     |████████████████████████████████| 31.2MB 137kB/s 
ERROR: umap-learn 0.4.3 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,)), ])
trainset = datasets.MNIST('./',download = True, train = True, transform = transform)
testset = datasets.MNIST('./',download= True, train= False, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64,shuffle = True)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle = True)

dataiter = iter(trainloader)
images, labels = dataiter.next()

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [ ]:
#define CNN source = https://zhenye-na.github.io/2018/09/28/pytorch-cnn-cifar10.html
#padding same 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

net = Net()
torch.save(net.state_dict(),'weights_i.pth')

In [ ]:
#training
#hiumang
def one_hot_embedding(labels, num_classes):
    """Embedding labels to one-hot form.

    Args:
      labels: (LongTensor) class labels, sized [N,].
      num_classes: (int) number of classes.

    Returns:
      (tensor) encoded labels, sized [N, #classes].
    """
    y = torch.eye(num_classes) 
    return y[labels] 



criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.003, momentum = 0.9)
for epoch in range(20):  # loop over the dataset multiple times
    correct = 0
    total = 0
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # labelsOH = one_hot_embedding(labels,10).long()
        # labelsOH = labelsOH.to(cuda)
        # labels = labels.to(cuda)
        # zero the parameter gradients
        optimizer.zero_grad()

#         # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()        

        # print statistics
        running_loss += loss.item()
    print("Epoch Number: "+str(epoch+1)+'Training Accuracy: %d %%' % (
    100 * correct / total))

    # print("Epoch Number: "+str(epoch+1)+" Loss is: "+str(running_loss / 60000))
    # print('[%d] loss: %.3f' %(epoch + 1), (running_loss / 60000))

#save learned weights
torch.save(net.state_dict(),'weights_f.pth')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch Number: 1Training Accuracy: 67 %
Epoch Number: 2Training Accuracy: 88 %
Epoch Number: 3Training Accuracy: 91 %
Epoch Number: 4Training Accuracy: 92 %
Epoch Number: 5Training Accuracy: 93 %
Epoch Number: 6Training Accuracy: 93 %
Epoch Number: 7Training Accuracy: 94 %
Epoch Number: 8Training Accuracy: 94 %
Epoch Number: 9Training Accuracy: 95 %
Epoch Number: 10Training Accuracy: 95 %
Epoch Number: 11Training Accuracy: 95 %
Epoch Number: 12Training Accuracy: 95 %
Epoch Number: 13Training Accuracy: 95 %
Epoch Number: 14Training Accuracy: 95 %
Epoch Number: 15Training Accuracy: 95 %
Epoch Number: 16Training Accuracy: 95 %
Epoch Number: 17Training Accuracy: 96 %
Epoch Number: 18Training Accuracy: 96 %
Epoch Number: 19Training Accuracy: 96 %
Epoch Number: 20Training Accuracy: 96 %


In [ ]:
#testing #10 epochs
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
acc = (correct/total)
np.save('model_accuracy_3.npy',acc)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 10000 test images: 96 %


In [ ]:
#vectorize wi
wi =torch.load('weights_i.pth')

wi1 = np.asarray(wi['conv1.weight'].resize(1,10*1*5*5))
wi2 = np.asarray(wi['conv2.weight'].resize(1,20*10*5*5))
wi3 = np.asarray(wi['fc1.weight'].resize(1,320*50))  #size of the resulting image after the given layers of convolutions 
wi4 = np.asarray(wi['fc2.weight'].resize(1,50*10))  #size of the resulting image after the given layers of convolutions 

wi_vec = np.concatenate((wi1,wi2,wi3,wi4),axis =1)
wi_vec = wi_vec.T
dim_w = wi_vec.shape[0]
wi_vec.ravel().squeeze()

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


array([ 0.14624207, -0.06318393,  0.19211279, ..., -0.09252968,
        0.12423135, -0.07830556], dtype=float32)

In [ ]:
#vectorize wf
wf  =torch.load('weights_f.pth')

wf1 = np.asarray(wf['conv1.weight'].resize(1,10*1*5*5))
wf2 = np.asarray(wf['conv2.weight'].resize(1,20*10*5*5))
wf3 = np.asarray(wf['fc1.weight'].resize(1,320*50))  #size of the resulting image after the given layers of convolutions 
wf4 = np.asarray(wf['fc2.weight'].resize(1,50*10))  #size of the resulting image after the given layers of convolutions 

wf_vec = np.concatenate((wf1,wf2,wf3,wf4),axis =1)
wf_vec = wf_vec.T
wf_vec.ravel().squeeze()

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


array([-0.07547412,  0.02723063,  0.3888827 , ..., -0.21531783,
        0.22056037, -0.31016245], dtype=float32)

In [ ]:
#initialize I run 1
from scipy import sparse
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384, 10*25,250+200*25,250+200*25+320*50,250+200*25+320*50+500])
num_rows_arr = np.sort(num_rows_arr)
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)

      w = np.zeros((dim_w,1))
      w[0:n_rows] = wf_vec[0:n_rows]
      w[n_rows:] = wi_vec[n_rows:]

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  320*50]
      w3 = torch.from_numpy(w3).reshape(50,320)
      w4 = w[10*1*5*5+20*10*5*5 +  320*50: 10*1*5*5+20*10*5*5 +  320*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.1135

 Dist from Wi = 0.239442814777908 , Dist from Wf =11.196396249075699

 n_rows = 4 , Accuracy =0.1135

 Dist from Wi = 0.372258101492695 , Dist from Wf =11.19276756364297

 n_rows = 8 , Accuracy =0.1137

 Dist from Wi = 0.5247500234508403 , Dist from Wf =11.186655409046686

 n_rows = 16 , Accuracy =0.1136

 Dist from Wi = 0.881425805213457 , Dist from Wf =11.16421561856313

 n_rows = 32 , Accuracy =0.1141

 Dist from Wi = 1.2609639613575456 , Dist from Wf =11.127739739768478

 n_rows = 64 , Accuracy =0.1205

 Dist from Wi = 2.0940969300799632 , Dist from Wf =11.001426265502795

 n_rows = 128 , Accuracy =0.1334

 Dist from Wi = 2.656429568557129 , Dist from Wf =10.879338388668701

 n_rows = 250 , Accuracy =0.137

 Dist from Wi = 3.327192679639078 , Dist from Wf =10.693288114533399

 n_rows = 256 , Accuracy =0.1344

 Dist from Wi = 3.3318929537163524 , Dist from Wf =10.69182450159188

 n_rows = 512 , Accuracy =0.1424

 Dist from Wi = 3.579048215885906 , Di

In [ ]:
np.save("accuracy_I3.npy", acc)
np.save("distance_i_I3.npy", dist_i)
np.save("distance_f_I3.npy", dist_f)

In [ ]:
#initialize Irev run 1
from scipy import sparse
den = 10/dim_w
num_rows_arr  = np.asarray([2,4,8,16,32,64,128,256,512,1024,2048,4096,8192, 16384, 10*25,250+200*25,250+200*25+320*50,250+200*25+320*50+500])
num_rows_arr = np.sort(num_rows_arr)
n_runs = 1
acc  = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_i = np.zeros((n_runs,num_rows_arr.shape[0]))
dist_f = np.zeros((n_runs,num_rows_arr.shape[0]))

for i_out in range(n_runs):
  print("outer iteration number:",(i_out+1))
  for i_in in range(num_rows_arr.shape[0]):
      n_rows = num_rows_arr[i_in]
      #minimize Z such that AZ = A(Wf-w_init)

      w = np.zeros((dim_w,1))
      w[0:-n_rows] = wi_vec[0:-n_rows] #check should be wi
      w[-n_rows:] = wf_vec[-n_rows:]   #check should be wf

      #test performance of this reinitialization
      w1= w[0:10*1*5*5]
      w1 = torch.from_numpy(w1).reshape( 10,1,5,5)
      w2 = w[10*1*5*5:10*1*5*5+20*10*5*5]
      w2 = torch.from_numpy(w2).reshape(20,10,5,5)
      w3 = w[10*1*5*5+20*10*5*5 : 10*1*5*5+20*10*5*5 +  320*50]
      w3 = torch.from_numpy(w3).reshape(50,320)
      w4 = w[10*1*5*5+20*10*5*5 +  320*50: 10*1*5*5+20*10*5*5 +  320*50 + 10*50]
      w4 = torch.from_numpy(w4).reshape(10,50)
      
      wf =torch.load('weights_f.pth')
      wf['conv1.weight'] = (w1)
      wf['conv2.weight'] = (w2)
      wf['fc1.weight'] = (w3)
      wf['fc2.weight'] = (w4)
      torch.save(wf,'w_affex.pth')


      net.load_state_dict(torch.load('w_affex.pth'))
      #testing
      correct = 0
      total = 0
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = net(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      # print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

      acc[i_out,i_in] = correct/total
      dist_i[i_out,i_in] = np.linalg.norm(w-wi_vec)
      dist_f[i_out,i_in] = np.linalg.norm(w-wf_vec)
      print("\n n_rows = {0} , Accuracy ={1}".format(n_rows,acc[i_out,i_in]))
      print("\n Dist from Wi = {0} , Dist from Wf ={1}".format(dist_i[i_out,i_in],dist_f[i_out,i_in])) 

outer iteration number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



 n_rows = 2 , Accuracy =0.1135

 Dist from Wi = 0.2510715051654662 , Dist from Wf =11.19614151961108

 n_rows = 4 , Accuracy =0.1135

 Dist from Wi = 0.29543091106656405 , Dist from Wf =11.195058838820435

 n_rows = 8 , Accuracy =0.1136

 Dist from Wi = 0.4596593442840514 , Dist from Wf =11.189518984973281

 n_rows = 16 , Accuracy =0.1137

 Dist from Wi = 0.7684126461621437 , Dist from Wf =11.172562993023725

 n_rows = 32 , Accuracy =0.115

 Dist from Wi = 1.0414074575625059 , Dist from Wf =11.150430141263536

 n_rows = 64 , Accuracy =0.1144

 Dist from Wi = 1.2578106291332223 , Dist from Wf =11.128096613936483

 n_rows = 128 , Accuracy =0.1176

 Dist from Wi = 1.8232063237745506 , Dist from Wf =11.049549336005231

 n_rows = 250 , Accuracy =0.119

 Dist from Wi = 2.7024045447671337 , Dist from Wf =10.86800954656772

 n_rows = 256 , Accuracy =0.1136

 Dist from Wi = 2.720547998385898 , Dist from Wf =10.863481965573637

 n_rows = 512 , Accuracy =0.1192

 Dist from Wi = 3.869531249153176

In [ ]:
np.save("accuracy_UI3.npy", acc)
np.save("distance_i_UI3.npy", dist_i)
np.save("distance_f_UI3.npy", dist_f)